# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here

import pandas as pd

data = pd.read_csv('../training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)
data.columns=['target','id','date','flag','user','text']

data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:
data.target.value_counts()

0    800000
4    800000
Name: target, dtype: int64

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
# your code here

import re
from nltk.tokenize import word_tokenize
# from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def clean_up(s):
    
    # removes urls beginning 'http' or 'www.'
    s = re.sub('\\bhttp[^ ]*','',s)
    s = re.sub('\\bwww.[^ ]*','',s)

    # removes special characters and numbers and replaces with space
    s = re.sub(r'[^a-zA-Z ]', ' ', s).lower()

    # makes lower case
    s = s.lower()

    return s

def tokenize(s):
    return word_tokenize(s)

# I will just use the lemmatizer component of this function:
def stem_and_lemmatize(list):
    # ps = PorterStemmer()
    lz = WordNetLemmatizer() 
    
    # stemmed = [ps.stem(word) for word in list]
    lemmatized = [lz.lemmatize(word) for word in list]

    return lemmatized # only want one not both

def remove_stopwords(l):
    without_sw = [word for word in l if not word in stopwords.words('english')]
    return without_sw

In [4]:
# take a sample of the dataset
sample=data.sample(n=20000,random_state=1234)
display(sample.shape)
sample.head()

(20000, 6)

,target,id,date,flag,user,text
1062048,4,1964017646,Fri May 29 13:35:57 PDT 2009,NO_QUERY,ashakur,@PcShakur I guess so. We should go make it a f...
1236895,4,1992914899,Mon Jun 01 09:31:47 PDT 2009,NO_QUERY,RabbiShaiSpecht,@PsychicSophie Thank you!
1237054,4,1992954642,Mon Jun 01 09:35:38 PDT 2009,NO_QUERY,RobinRene7,it's just another day here at the office...I a...
229979,0,1978627799,Sat May 30 23:57:09 PDT 2009,NO_QUERY,valfvergara,@triciasays i miss kickboxing
760772,0,2296927988,Tue Jun 23 09:48:35 PDT 2009,NO_QUERY,SarahAnnGreen,@AmandaFClark Thats rubbish!


In [5]:
%%time

clean = sample['text'].apply(clean_up)

token = clean.apply(tokenize)

lemmatized = token.apply(stem_and_lemmatize)

text_processed = lemmatized.apply(remove_stopwords)

CPU times: total: 1min 4s
Wall time: 1min 4s


In [6]:
sample['text_processed'] = text_processed

sample.head()

,target,id,date,flag,user,text,text_processed
1062048,4,1964017646,Fri May 29 13:35:57 PDT 2009,NO_QUERY,ashakur,@PcShakur I guess so. We should go make it a f...,"[pcshakur, guess, go, make, family, affair]"
1236895,4,1992914899,Mon Jun 01 09:31:47 PDT 2009,NO_QUERY,RabbiShaiSpecht,@PsychicSophie Thank you!,"[psychicsophie, thank]"
1237054,4,1992954642,Mon Jun 01 09:35:38 PDT 2009,NO_QUERY,RobinRene7,it's just another day here at the office...I a...,"[another, day, office, ready, tonight, back, p..."
229979,0,1978627799,Sat May 30 23:57:09 PDT 2009,NO_QUERY,valfvergara,@triciasays i miss kickboxing,"[triciasays, miss, kickboxing]"
760772,0,2296927988,Tue Jun 23 09:48:35 PDT 2009,NO_QUERY,SarahAnnGreen,@AmandaFClark Thats rubbish!,"[amandafclark, thats, rubbish]"


In [ ]:
# sample.to_csv('sample.csv')

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [7]:
# consolidate these all back into a "blob"
def re_blob(row):
    return " ".join(row['text_processed'])
sample['clean_blob'] = sample.apply(re_blob,axis=1)

sample.head()

,target,id,date,flag,user,text,text_processed,clean_blob
1062048,4,1964017646,Fri May 29 13:35:57 PDT 2009,NO_QUERY,ashakur,@PcShakur I guess so. We should go make it a f...,"[pcshakur, guess, go, make, family, affair]",pcshakur guess go make family affair
1236895,4,1992914899,Mon Jun 01 09:31:47 PDT 2009,NO_QUERY,RabbiShaiSpecht,@PsychicSophie Thank you!,"[psychicsophie, thank]",psychicsophie thank
1237054,4,1992954642,Mon Jun 01 09:35:38 PDT 2009,NO_QUERY,RobinRene7,it's just another day here at the office...I a...,"[another, day, office, ready, tonight, back, p...",another day office ready tonight back playing ...
229979,0,1978627799,Sat May 30 23:57:09 PDT 2009,NO_QUERY,valfvergara,@triciasays i miss kickboxing,"[triciasays, miss, kickboxing]",triciasays miss kickboxing
760772,0,2296927988,Tue Jun 23 09:48:35 PDT 2009,NO_QUERY,SarahAnnGreen,@AmandaFClark Thats rubbish!,"[amandafclark, thats, rubbish]",amandafclark thats rubbish


In [8]:
# count occurence of all unique words in the 20,000 "blobs"
from sklearn.feature_extraction.text import CountVectorizer
bow_vect = CountVectorizer()
bow_vect.fit(sample.clean_blob)

CountVectorizer()

In [9]:
# and put them in a dataframe

bow_df = pd.DataFrame(bow_vect.transform(sample['clean_blob']).toarray(),
                    columns=bow_vect.get_feature_names_out())

bow_df.head()

,aa,aaaa,aaaaa,aaaaaaaaaaa,aaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaargh,aaaaaaaaaaarggggggggggghhhhhhhhh,aaaaaaaahh,aaaaaand,...,zuzubrandy,zvyknu,zwinky,zyber,zyleeth,zzz,zzzz,zzzzz,zzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# bow_vect.vocabulary_

In [10]:
# summing each column will give the total frequency of words from all blobs
word_frequencies = bow_df.sum()
word_frequencies

aa                             2
aaaa                           2
aaaaa                          1
aaaaaaaaaaa                    1
aaaaaaaaaaaaa                  1
                              ..
zzz                            4
zzzz                           4
zzzzz                          1
zzzzzzz                        1
zzzzzzzzzzzzzzzzzzzzzzzzzzz    1
Length: 26419, dtype: int64

In [11]:
# choose just the top 5000 words
top_5000_words = word_frequencies.sort_values(ascending=False).head(5000)

In [13]:
# and filter our bag of words dataframe to just get columns for these 5000
# df shape is 20000 samples of text across 5000 words
top_5000_words_df = bow_df[top_5000_words.index]
display(top_5000_words_df.shape)
top_5000_words_df.head()

(20000, 5000)

,day,wa,good,get,go,like,quot,got,love,time,...,trigger,trio,fill,tripping,tru,filet,anatomy,tuck,fiction,tuition
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# realise I could have just used this earlier:
# bow_vect = CountVectorizer(max_features=5000)
# DOH!!!

In [15]:
display(len(sample.target))
sample.target[0:5]

20000

1062048    4
1236895    4
1237054    4
229979     0
760772     0
Name: target, dtype: int64

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [17]:
# your code here
words = top_5000_words_df.columns
len(words)

5000

In [ ]:
# list(words)

In [18]:
top_5000_words_df_t = top_5000_words_df.replace({0:False, 1:True})
top_5000_words_df_t.head()

,day,wa,good,get,go,like,quot,got,love,time,...,trigger,trio,fill,tripping,tru,filet,anatomy,tuck,fiction,tuition
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# I built the following function with a small section of the dataframe
# top_5000_words_df_t.iloc[:3,:5]

In [ ]:
# top_5000_words_df_t.day.iloc[2]

In [19]:
from tqdm import tqdm

In [20]:
%%time

list_of_dictionaries = []

for row in tqdm( range(len(sample.target)) ): # all 20000 rows
    row_dict = {}
    for word in list(words): # columns in top_5000_words_df_t
        row_dict[word] = top_5000_words_df_t[word].iloc[row]
    list_of_dictionaries.append(row_dict)
len(list_of_dictionaries)

100%|█████████████████████████████████████████████████████████████████████████| 20000/20000 [19:49<00:00, 16.82it/s]

CPU times: total: 19min 45s
Wall time: 19min 49s


20000

In [21]:
list_of_dictionaries[2]['day']

True

In [22]:
len(list_of_dictionaries[2])

5000

In [23]:
target_list = sample.target.replace({0:False, 4:True})

feature_set = list(zip(list_of_dictionaries, target_list))

In [ ]:
# feature_set[0:2]

In [24]:
sample.target.iloc[0:5]

1062048    4
1236895    4
1237054    4
229979     0
760772     0
Name: target, dtype: int64

In [25]:
display(feature_set[0][1])
display(feature_set[1][1])
display(feature_set[2][1])
display(feature_set[3][1])
display(feature_set[4][1])

True

True

True

False

False

In [ ]:
# it has been transformed as needed

### Building and Training Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [26]:
# take the first 80% of rows as training and the last 20% as validation

training_index = int(len(feature_set)*0.8)

train_data = feature_set[:training_index] # first 80%

test_data = feature_set[training_index:] # last 20%

len(train_data)

16000

In [27]:
from nltk import NaiveBayesClassifier

In [28]:
%%time

NBC = NaiveBayesClassifier.train(train_data)

CPU times: total: 1min 11s
Wall time: 1min 11s


In [31]:
NBC.show_most_informative_features(50)

Most Informative Features
                horrible = True            False : True   =     19.2 : 1.0
                    sick = True            False : True   =     18.8 : 1.0
            disappointed = True            False : True   =     17.2 : 1.0
                   worst = True            False : True   =     17.2 : 1.0
                     vip = True             True : False  =     16.8 : 1.0
                 welcome = True             True : False  =     12.8 : 1.0
                  broken = True            False : True   =     12.8 : 1.0
                headache = True            False : True   =     12.3 : 1.0
                     fml = True            False : True   =     11.8 : 1.0
               excellent = True             True : False  =     11.5 : 1.0
                 missing = True            False : True   =     11.3 : 1.0
                  lonely = True            False : True   =     11.1 : 1.0
                    hurt = True            False : True   =     11.0 : 1.0

In [ ]:
# very strong correlation on some words to suggest negativity
# mainly relating to illness, death, absence and so on

# likewise the strongest postive correlation relates to thanking someone
# also welcoming, congratulating, and positive superlatives like "excellent"

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [30]:
%%time

from nltk import classify

print("NBC model accuracy: ", classify.accuracy(NBC, test_data))

NBC model accuracy:  0.7255
CPU times: total: 48.5 s
Wall time: 48.8 s


In [ ]:
# accuracy only 72.5% so a good model, however still space for improvement

In [ ]:
# in the top 5000 words, we still see "got", "get", "wa", "go"
# these words are unlikely to add much value to the model
# so trimming columns might help reduce noise

# preparing my list_of_dictionaries took 20 minutes, so we could improve the model
# by including 10x more training rows and running this overnight

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here